In [ ]:
import zipfile
from bs4 import BeautifulSoup
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from text_unidecode import unidecode
import random, pandas as pd
from datetime import datetime, timedelta
import datetime, time, gspread
from google.oauth2.service_account import Credentials
import csv, re, os, json
from deep_translator import GoogleTranslator
from docx import Document
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

In [ ]:
ethiopic_transliteration = {
    'ሀ': 'ha', 'ሁ': 'hu', 'ሂ': 'hi', 'ሃ': 'ha', 'ሄ': 'hé', 'ህ': 'h', 'ሆ': 'ho',
    'ሐ': 'ha', 'ሑ': 'hu', 'ሒ': 'hi', 'ሓ': 'ha', 'ሔ': 'hé', 'ሕ': 'h', 'ሖ': 'ho', 'ሗ': 'hwa',
    'ኀ': 'ha', 'ኁ': 'hu', 'ኂ': 'hi', 'ኃ': 'ha', 'ኄ': 'hé', 'ኅ': 'h', 'ኆ': 'ho',
    'ለ': 'le', 'ሉ': 'lu', 'ሊ': 'li', 'ላ': 'la', 'ሌ': 'lé', 'ል': 'l', 'ሎ': 'lo', 'ሏ': 'la',
    'መ': 'mä', 'ሙ': 'mu', 'ሚ': 'mi', 'ማ': 'ma', 'ሜ': 'mé', 'ም': 'm', 'ሞ': 'mo', 'ሟ': 'mwa',
    'ሠ': 'se', 'ሡ': 'su', 'ሢ': 'si', 'ሣ': 'sa', 'ሤ': 'sé', 'ሥ': 's', 'ሦ': 'so', 'ሧ': 'swa',
    'ሰ': 'se', 'ሱ': 'su', 'ሲ': 'si', 'ሳ': 'sa', 'ሴ': 'sé', 'ስ': 's', 'ሶ': 'so', 'ሷ': 'swa',
    'ረ': 're', 'ሩ': 'ru', 'ሪ': 'ri', 'ራ': 'ra', 'ሬ': 'ré', 'ር': 'r', 'ሮ': 'ro', 'ሯ': 'rwa',
    'ሸ': 'she', 'ሹ': 'shu', 'ሺ': 'shi', 'ሻ': 'sha', 'ሼ': 'shé', 'ሽ': 'sh', 'ሾ': 'sho', 'ሿ': 'shwa',
    'ቀ': "k'", 'ቁ': "k'u", 'ቂ': "k'i", 'ቃ': "k'a", 'ቄ': "k'é", 'ቅ': "k'", 'ቆ': "k'o",
    'በ': 'be', 'ቡ': 'bu', 'ቢ': 'bi', 'ባ': 'ba', 'ቤ': 'bé', 'ብ': 'b', 'ቦ': 'bo', 'ቧ': 'bwa',
    'ተ': 'te', 'ቱ': 'tu', 'ቲ': 'ti', 'ታ': 'ta', 'ቴ': 'té', 'ት': 't', 'ቶ': 'to', 'ቷ': 'twa',
    'ቸ': 'tche', 'ቹ': 'tchu', 'ቺ': 'tchi', 'ቻ': 'tcha', 'ቼ': 'tché', 'ች': 'tch', 'ቾ': 'tcho', 'ቿ': 'tchwa',
    'ጨ': "tch'a", 'ጩ': "tch'u", 'ጪ': "tch'i", 'ጫ': "tch'a", 'ጬ': "tch'é", 'ጭ': "tch'", 'ጮ': "tch'o", 'ጯ': "tch'wa",
    'ነ': 'na', 'ኑ': 'nu', 'ኒ': 'ni', 'ና': 'na', 'ኔ': 'né', 'ን': 'n', 'ኖ': 'no', 'ኗ': 'nwa',
    'ኘ': 'ñe', 'ኙ': 'ñu', 'ኚ': 'ñi', 'ኛ': 'ñ', 'ኜ': 'ñé', 'ኝ': 'ñ', 'ኞ': 'ño',
    'አ': 'a', 'ኡ': 'u', 'ኢ': 'i', 'ኣ': 'a', 'ኤ': 'é', 'እ': 'ē', 'ኦ': 'o', 'ኧ': 'a',
    'ከ': 'ka', 'ኩ': 'ku', 'ኪ': 'ki', 'ካ': 'ka', 'ኬ': 'ké', 'ክ': 'k', 'ኮ': 'ko', 'ኯ': 'kwē',
    'ኸ': 'he', 'ኹ': 'hu', 'ኺ': 'hi', 'ኻ': 'ha', 'ኼ': 'hé', 'ኽ': 'h', 'ኾ': 'ho', 'ዀ': 'hwē',
    'ወ': 'wa', 'ዉ': 'wu', 'ዊ': 'wi', 'ዋ': 'wa', 'ዌ': 'wé', 'ው': 'w', 'ዎ': 'wo', 'ዏ': 'wē',
    'ዐ': 'a', 'ዑ': 'hu', 'ዒ': 'i', 'ዓ': 'a', 'ዔ': 'é', 'ዕ': 'ē', 'ዖ': 'o',
    'ዘ': 'za', 'ዙ': 'zu', 'ዚ': 'zi', 'ዛ': 'za', 'ዜ': 'zé', 'ዝ': 'z', 'ዞ': 'zo', 'ዟ': 'zwa',
    'ዠ': 'je', 'ዡ': 'ju', 'ዢ': 'ji', 'ዣ': 'ja', 'ዤ': 'jé', 'ዥ': 'j', 'ዦ': 'jo', 'ዧ': 'jwa',
    'የ': 'ye', 'ዩ': 'yu', 'ዪ': 'yi', 'ያ': 'ya', 'ዬ': 'yé', 'ይ': 'y', 'ዮ': 'yo', 'ዯ': 'yē',
    'ደ': 'de', 'ዱ': 'du', 'ዲ': 'di', 'ዳ': 'da', 'ዴ': 'dé', 'ድ': 'd', 'ዶ': 'do', 'ዷ': 'dwa',
    'ጀ': 'dje', 'ጁ': 'dju', 'ጂ': 'dji', 'ጃ': 'dja', 'ጄ': 'djé', 'ጅ': 'dj', 'ጆ': 'djo', 'ጇ': 'djwa',
    'ገ': 'g', 'ጉ': 'gu', 'ጊ': 'gi', 'ጋ': 'ga', 'ጌ': 'gé', 'ግ': 'g', 'ጎ': 'go', 'ጏ': 'gē',
    'ጠ': "t'a", 'ጡ': "t'u", 'ጢ': "t'i", 'ጣ': "t'a", 'ጤ': "t'é", 'ጥ': "t'", 'ጦ': "t'o", 'ጧ': "t'wa",
    'ጰ': "s'a", 'ጱ': "s'u", 'ጲ': "s'i", 'ጳ': "s'a", 'ጴ': "s'é", 'ጵ': "s'", 'ጶ': "s'o", 'ጷ': "s'wa",
    'ጸ': "ts'e", 'ጹ': "ts'u", 'ጺ': "ts'i", 'ጻ': "ts'a", 'ጼ': "ts'é", 'ጽ': "ts'", 'ጾ': "ts'o", 'ጿ': "ts'wa",
    'ፀ': 'tse', 'ፁ': 'tsu', 'ፂ': 'tsi', 'ፃ': 'tsa', 'ፄ': 'tsé', 'ፅ': 'ts', 'ፆ': 'tso',
    'ፈ': 'fe', 'ፉ': 'fu', 'ፊ': 'fi', 'ፋ': 'fa', 'ፌ': 'fé', 'ፍ': 'f', 'ፎ': 'fo', 'ፏ': 'fwa',
    'ፐ': 'pe', 'ፑ': 'pu', 'ፒ': 'pi', 'ፓ': 'pa', 'ፔ': 'pé', 'ፕ': 'p', 'ፖ': 'po'
}


def transliterate_ethiopic(text):
    return ''.join(ethiopic_transliteration.get(char, char) for char in text)

In [ ]:
# Définir les scopes requis pour accéder à Google Sheets et Google Drive
scopes = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/drive.file',
    'https://www.googleapis.com/auth/drive.appdata','https://www.googleapis.com/auth/drive.appfolder'
]

# Chemin vers le fichier de crédentials JSON
path_to_google_cred = "C:/Users/flosr/Engineering/Data Engineering/credentials.json"

# Créer les crédentials à partir du fichier de service account
creds = Credentials.from_service_account_file(path_to_google_cred, scopes=scopes)

# Autoriser l'accès en utilisant les crédentials
client = gspread.authorize(creds)

# ID de la feuille Google Sheets
sheet_id = "179ajVaPhOWouRJTP19vIfSD3Q2FJCnZiOQtj2BScfn0"

# Ouvrir la feuille Google Sheets en utilisant son ID
spreadsheet = client.open_by_key(sheet_id)

worksheets = spreadsheet.worksheets()

drive_service = build('drive', 'v3', credentials=creds)


In [ ]:
def show_sheet_titles():
    worksheet_titles = [ws.title for ws in worksheets]
    return worksheet_titles

In [ ]:
Persan = Russe = Amharic = None
worksheets_list = []

for ws in worksheets:
    worksheets_list.append(ws.title)
    if ws.title == 'Persan':
        Persan = ws
    if ws.title == 'Russe':
        Russe = ws
    if ws.title == 'Amharic':
        Amharic = ws       
    if ws.title == 'Somali':
        Somali = ws       
    if ws.title == 'Arabic':
        Arabic = ws
    if ws.title == 'Tigrinya':
        Tigrinya = ws
    if ws.title == "Amharic Alphabet":
        Amharic_Alphabet = ws
        
        break    

In [ ]:
sheet_title_basic =['_CONCEPTS_DE_BASE',
 "L'HOMME",
 'VÊTEMENTS ET ACCESSOIRES',
 'LES_ALIMENTS',
 'FAMILLE_ET_AMIS',
 'CARACTERE_ET_EMOTIONS',
 'MEDECINE',
 "L'HABITAT",
 'ACTIVITES_HUMAINES',
 'TECHNIQUE_TRANSPORTS',
 'GENS_ET_EVENEMENTS',
 'LA_NATURE']

In [78]:
def create_sheets_for_language(sheet_lang):
    if sheet_lang == "Persian" or sheet_lang == "Persan":
        lang = "Persian"
        abrev_lang = "Persian"
    if sheet_lang == "Hindi" or sheet_lang == "hindi":
        abrev_lang = "hi"
    if sheet_lang == "Indonesian" or sheet_lang == "Persan":
        abrev_lang = "Ind"

    spreadsheet = client.open_by_key(sheet_id)

    header = ["FR/ENG", abrev_lang, "Translittération", "V"] * 12
    NUMBER = 0
    for title in sheet_title_basic:
        NUMBER +=1 
        new_title = f"{sheet_lang}_{NUMBER}_{title}"
        sheet = spreadsheet.add_worksheet(title=new_title, rows="100", cols="26")

        # Mise à jour de la première ligne avec le header et formatage
        cell_range_header = "A1:AZ1"
        sheet.update(cell_range_header, [header])

        # Mise à jour de la deuxième ligne avec les mêmes données et formatage
        cell_range_subheader = "A2:AZ2"
        sheet.update(cell_range_subheader, [header])

        # Définir le formatage pour les deux premières lignes
        first_two_rows_format = {
            "backgroundColor": {"blue": 0.392, "green": 0.584, "red": 0.929},  # Cornflower blue
            "textFormat": {"bold": True, "foregroundColor": {"red": 1.0, "green": 1.0, "blue": 1.0}}  # Blanc
        }

        # Appliquer le formatage aux deux premières lignes
        format_cell_ranges = [
            {
                "range": {
                    "sheetId": sheet.id,
                    "startRowIndex": 0,
                    "endRowIndex": 2,  # Inclut la première et la deuxième ligne
                    "startColumnIndex": 0,
                    "endColumnIndex": len(header)
                },
                "cell": {
                    "userEnteredFormat": first_two_rows_format
                },
                "fields": "userEnteredFormat(backgroundColor,textFormat)"
            }
        ]

        # Envoyer la requête de mise à jour de format en batch
        body = {"requests": [{"repeatCell": range} for range in format_cell_ranges]}
        spreadsheet.batch_update(body)

        # Assigner la feuille à une variable globale avec un nom propre
        globals()[title.replace(" ", "_")] = sheet

# Exemple d'utilisation
sheet_lang = "Persian"
create_sheets_for_language(sheet_lang)

C:\Users\flosr\AppData\Local\Temp\ipykernel_1672\700653075.py:12: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update(cell_range, [header])


In [79]:
def delete_worksheet_by_name(keyword):
    spreadsheet = client.open_by_key(sheet_id)
    worksheets = spreadsheet.worksheets()

    for worksheet in worksheets:
        worksheet_name = worksheet.title
        if keyword in worksheet.title:
            # Supprimer la feuille de calcul
            spreadsheet.del_worksheet(worksheet)
            print(f"Worksheet '{worksheet_name}' deleted.")
            

delete_worksheet_by_name("Persian")


Worksheet 'Persian__CONCEPTS_DE_BASE' deleted.
Worksheet 'Persian__L'HOMME' deleted.
Worksheet 'Persian__VÊTEMENTS ET ACCESSOIRES' deleted.
Worksheet 'Persian__LES_ALIMENTS' deleted.
Worksheet 'Persian__FAMILLE_ET_AMIS' deleted.
Worksheet 'Persian__CARACTERE_ET_EMOTIONS' deleted.
Worksheet 'Persian__MEDECINE' deleted.
Worksheet 'Persian__L'HABITAT' deleted.
Worksheet 'Persian__ACTIVITES_HUMAINES' deleted.
Worksheet 'Persian__TECHNIQUE_TRANSPORTS' deleted.
Worksheet 'Persian__GENS_ET_EVENEMENTS' deleted.
Worksheet 'Persian__LA_NATURE' deleted.


In [ ]:
show_sheet_titles()

Création des Sheets Russian

In [ ]:
# Exemple d'utilisation
sheet_lang = "Russian"
create_sheets_for_language(sheet_lang)

Création des Sheets Hindi

In [ ]:
# Exemple d'utilisation
sheet_lang = "Hindi"
create_sheets_for_language(sheet_lang)

Reference Sheet Creation

In [94]:
import time

def get_sheet_content(sheet_title):
    # Ouvrir la feuille de calcul
    spreadsheet = client.open_by_key(sheet_id)

    try:
        # Accéder à la feuille par son titre
        sheet = spreadsheet.worksheet(sheet_title)

        # Récupérer tout le contenu de la feuille
        sheet_content = sheet.get_all_values()

        return sheet_content

    except gspread.exceptions.WorksheetNotFound:
        print(f"Feuille '{sheet_title}' non trouvée.")
        return None

def create_reference_sheet(content_to_copy, sheet_title_to_create):
    # Ouvrir la feuille de calcul
    spreadsheet = client.open_by_key(sheet_id)

    # Vérifier si la feuille existe déjà
    try:
        spreadsheet.worksheet(sheet_title_to_create)
        print(f"Feuille '{sheet_title_to_create}' existe déjà. Ignorer la création.")
        return
    except gspread.exceptions.WorksheetNotFound:
        pass

    # Créer une nouvelle feuille avec le titre spécifié
    new_sheet = spreadsheet.add_worksheet(title=sheet_title_to_create, rows="100", cols="26")

    # Écrire le contenu dans la nouvelle feuille
    for row_idx, row in enumerate(content_to_copy, start=1):
        new_sheet.insert_row(row, index=row_idx)
        time.sleep(1.8)  # Attendre 1.8 secondes après chaque insertion de ligne

    print(f"Feuille '{sheet_title_to_create}' créée avec succès.")


In [92]:
CONCEPTS_DE_BASE = get_sheet_content("Amharic_CONCEPTS_DE_BASE")

In [95]:
create_reference_sheet(CONCEPTS_DE_BASE, "PROTOTYPE_CONCEPTS_DE_BASE")


Feuille 'PROTOTYPE_CONCEPTS_DE_BASE' créée avec succès.
